In [0]:
import requests
import re
import json
from datetime import datetime
from pyspark.sql import functions as F
import urllib.parse

CATALOG = "ptd_dev"
BASE = "https://api.gdeltproject.org/api/v2/doc/doc"

spark.sql(f"""
CREATE TABLE IF NOT EXISTS {CATALOG}.bronze.gdelt_tonechart (
    company_name STRING,
    date STRING,
    bin LONG,
    count LONG,
    ingestion_ts TIMESTAMP
)
USING DELTA
""")

In [0]:
def cleaned_gdelt_json_response(response):
    # Preprocesar el texto para eliminar el campo toparts y restos de title/url/texto suelto
    cleaned_text = re.sub(r',?\s*"toparts"\s*:\s*\[.*?\]', '', response.text, flags=re.DOTALL)
    cleaned_text = re.sub(r',?\s*\{\s*"url"\s*:\s*"[^"]*"\s*,\s*"title"\s*:\s*"[^"]*"\s*\}', '', cleaned_text)
    cleaned_text = re.sub(r',\s*,', ',', cleaned_text)
    cleaned_text = re.sub(r',\s*([\]\}])', r'\1', cleaned_text)
    cleaned_text = re.sub(r'("count"\s*:\s*\d+)[^},]*', r'\1', cleaned_text)
    cleaned_text = re.sub(r'("count"\s*:\s*\d+)\s*bin', r'\1},{"bin', cleaned_text)
    cleaned_text = re.sub(r',?\s*"title"\s*:\s*"[^"]*"', '', cleaned_text)
    cleaned_text = re.sub(r',?\s*"url"\s*:\s*"[^"]*"', '', cleaned_text)
    cleaned_text = re.sub(r'\}\]\s*\},\s*\{', '}, {', cleaned_text)
    cleaned_text = re.sub(r'\}\]\s*\}', '}', cleaned_text)
    cleaned_text = re.sub(r'"tonechart"\s*:\s*\[([^]]*)\][^}]*}', r'"tonechart": [\1]}', cleaned_text)
    matches = re.findall(r'\{ *\"bin\" *: *-?\d+ *, *\"count\" *: *\d+ *\}', cleaned_text)
    cleaned_text = '{"tonechart": [' + ','.join(matches) + ']}'
    if not cleaned_text.strip().endswith(']}'):
        cleaned_text = cleaned_text.strip() + ']}'
    return cleaned_text

def gdelt_gkg_url(company_alias: list[str], ts: str) -> str:
    # Construye la parte de la query con los alias
    if len(company_alias) > 1: 
        query = " OR ".join([f'"{alias}"' for alias in company_alias])
        query = f"({query})"
    else:
        query = f'"{company_alias[0]}"'
    return f"{BASE}?query={urllib.parse.quote_plus(query)}&ts={ts}&mode=tonechart&format=json"


In [0]:
rows, downloaded, skipped, errors = [], 0, 0, []

companies = spark.table(f"{CATALOG}.bronze.universe_sp100_snapshot").filter("active_flag = 1").collect()

for c in companies:
    try:
        df = spark.table(f"{CATALOG}.bronze.gdelt_tonechart").filter(
            (F.col("company_name") == c.company_name) &
            (F.col("date") == datetime.now().strftime("%Y-%m-%d"))
        )
        if df.count() == 0:
            aliases = []
            if " - " in c.company_alias:
                aliases = c.company_alias.split(" - ")
            else:
                aliases.append(c.company_alias)
            url = gdelt_gkg_url(aliases, "24hs")

            response = requests.get(url)
            response.raise_for_status()
            date = datetime.now().strftime("%Y-%m-%d")

            cleaned_text = cleaned_gdelt_json_response(response)
                
            data = json.loads(cleaned_text)
            for bin_info in data.get('tonechart', []):
                bin_value = bin_info.get('bin')
                count_value = bin_info.get('count')
                rows.append({
                    "company_name": c.company_name,
                    "date": date,
                    "bin": bin_value,
                    "count": count_value
                })
            if len(rows) > 0:
                downloaded+=1
            else:
                errors.append((c.company_name, "No data"))
                skipped += 1
        else:
            skipped += 1   
    except Exception as e:
        errors.append((c.company_name, str(e)))
        skipped += 1


In [0]:
if(rows != []):
    df = spark.createDataFrame(rows)
    df = df.withColumn("ingestion_ts", F.current_timestamp())
    df.write.mode("append").saveAsTable(f"{CATALOG}.bronze.gdelt_tonechart")
    
print(f"Descargados: {downloaded} | Saltados: {skipped} | Errores: {len(errors)}")
if errors:
    display(spark.createDataFrame(errors))